In [ ]:
from __future__ import print_function

In [ ]:
import sys
sys.path.insert(0, '../src/')
import generator

import numpy as np
import cplex

# Instance generator for Demand-Response problems

Generates instances of residential Demand Response problems, formulated as Mixed-Integer Linear Programs (MILP).

These models are described in Anjos, Lodi, Tanneau, _A Decentralized Framework for the Optimal Coordination of Distributed Energy Resources_


The considered problems are MILPs of the form:

$$
\begin{array}{rrl}
\displaystyle \min_{y, x} &  q^{T}y + \sum_{r} c_{r}^{T} x_{r}\\
s.t.  &  My + \sum_{r} A_{r} x_{r} &= b\\
& x_{r} &\in \mathcal{X}_{r}
\end{array}
$$
where $x_{r} \in \mathcal{X}_{r}$ encodes linear constraints and integrality requirements on $x_{r}$.

# Instances used in [Anjos et al]

This will generate the testbed that was used in [Anjos, Lodi, Tanneau].

For an example, see the `example.ipynb` notebook.

## Hyper-parameters

First, we generate a list of hyper-parameters for the instance generator. Instances can then be generated and solved individually.

In [ ]:
# Hyper-parameters values
N = [1024, 1536, 2048, 3072, 4096, 6144, 8192]  # number of households
T = [24, 48, 72, 96]  # time-horizon
P = [0.00, 0.33, 0.66, 1.00]  # proportion of renewables
n_runs = 10  # generate several instances for each value of (n, t, p)

In [ ]:
# generate list of hyper parameters
h = []

np.random.seed(0)
for n in N:
    for t in T:
        for p in P:
            for r in range(n_runs):
                s = np.random.randint(low=0, high=2**24)
                h.append((n, t, p, r, s))

## Generate instances

In [ ]:
for (n, t, p, r, s) in h:
    own_rates = {
        'pv': p,  # rooftop solar PV
        'dishwasher': 0.65,
        'clothes_washer': 0.9,
        'clothes_dryer': 0.75,
        'ev': p,  # electric vehicle
        'heating': 0.6,  # space heating
        'battery': p  # home battery
    }
    # generate instance
    m = generator.generate_instance(n, 408, t, own_rates, seed=s)
    
    # instance can now be solved, exported in an MPS / LP file...